In [20]:
import pandas as pd
import copy

In [21]:
keyname = 'Nc'
path = '/Users/dongjiacheng/Desktop/coder/mtd/RNA-seq/'+ keyname +'/'
print(path)

/Users/dongjiacheng/Desktop/coder/mtd/RNA-seq/Nc/


复杂的、糟糕的、清理数据

In [22]:
df_fpkm = pd.read_csv('./Nc/Nc_1_FPKM.csv', sep=',').copy()
df_fpkm

,Geneid,Chr,Start,End,Strand,Length,SRR10078920,SRR10078921,SRR1025973,SRR1025974,...,SRR9214968,SRR959443,SRR959444,SRR9601987,SRR9601988,SRR999602,SRR999604,SRR999605,SRR999606,SRR999607
0,NCU10129,NC_026501.1,1151,2878,-,1728,21.84,20.15,0.47,0.63,...,6.81,2.28,36.58,0.71,0.86,6.13,7.55,3.87,1.91,3.10
1,NCU09901,NC_026501.1,3178,6443,+,3266,7.08,9.90,15.15,15.25,...,11.44,10.97,30.22,0.66,1.48,11.97,12.40,8.75,11.83,10.59
2,NCU09903,NC_026501.1,9177,13334,+,4158,0.73,2.16,42.54,47.06,...,68.48,26.35,35.37,0.97,2.09,15.79,15.86,11.41,30.68,26.72
3,NCU11134,NC_026501.1,15930,16647,+,718,0.00,0.00,0.00,0.12,...,0.38,0.34,20.66,0.00,0.13,0.13,0.20,0.00,0.19,0.26
4,NCU09904,NC_026501.1,17441,19615,+,2175,8.12,14.51,0.34,0.73,...,0.96,0.34,12.31,0.38,0.17,18.59,20.32,10.27,1.22,1.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10585,NCU16025,NC_026614.1,55087,57247,+,2161,5.12,4.89,4.10,5.64,...,0.43,1.99,237.59,3.66,7.15,4.54,4.02,4.82,2.29,4.81
10586,NCU16026,NC_026614.1,55087,56529,+,1443,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
10587,NCU16027,NC_026614.1,58608,58832,+,225,58.73,38.26,7.17,21.87,...,1.22,33.38,1184.66,8.69,13.00,59.41,40.75,30.80,49.71,79.21
10588,NCU16028,NC_026614.1,59529,60281,+,753,58.04,51.44,6.82,14.40,...,2.70,30.90,757.43,13.13,23.55,65.50,53.66,28.06,43.04,75.57


In [23]:
df_sample = pd.read_excel(path + keyname + "_sample_raw_2023.12.18.xlsx").copy()
df_sample_run = df_sample[['Run']].copy()
df_sample_run

,Run
0,SRR400635
1,SRR400636
2,SRR400637
3,SRR400638
4,SRR400639
...,...
864,SRR22543627
865,SRR22543626
866,SRR22543625
867,SRR22543624


In [25]:
# 如果df_fpkm中SRR开头的列名在df_sample_run中，则保留，不要删除第一列
df_fpkm_filter = df_fpkm[df_fpkm.columns.intersection(df_sample_run['Run'])].copy()

# 将df_fpkm的第一列与df_fpk_filter拼接
df_fpkm_filter = pd.concat([df_fpkm.iloc[:,0], df_fpkm_filter], axis=1)

# 保存为csv
df_fpkm_filter.to_csv(path + keyname + '_1_FPKM_filter.csv', index=False, sep=',')

df_fpkm_filter

1. 将表达矩阵中的SRR号码，根据表格关系，替换为GSM号码

In [27]:
df_exp = pd.read_csv(path + keyname + "_1_FPKM_filter.csv").copy()
df_exp = df_exp.drop(df_exp.columns[1:6], axis=1) # df_exp不保存第二列到第六列

df_sample = pd.read_excel(path + keyname + "_sample_raw_2023.12.18.xlsx").copy()

# SRR 号码映射为 GSM 号码
id_map = dict(zip(df_sample["Run"], df_sample["Sample Name"]))
df_gsm = pd.DataFrame(df_exp)

# 替换文件中的样本名称
new_columns = ["Geneid"] + [id_map[col] for col in df_exp.columns[1:]]
df_exp.columns = new_columns

df_gsm = pd.DataFrame(df_exp)
df_gsm.to_csv(path + keyname + "_4_FPKM_GSM.csv", sep=',',index=False)
df_gsm

,Geneid,GSM1260337,GSM1262788,GSM1262789,GSM1262790,GSM1262791,GSM1262792,GSM1262793,GSM1262794,GSM1262795,...,GSM3861097,GSM1219411,GSM1219412,GSM3905561,GSM3905562,GSM1238597,GSM1238602,GSM1238604,GSM1238606,GSM1238609
0,NCU10129,4.44,16.40,15.10,15.68,13.17,10.76,10.18,10.72,10.33,...,6.81,2.28,36.58,0.71,0.86,6.13,7.55,3.87,1.91,3.10
1,NCU09901,3.92,4.50,5.95,4.87,4.55,6.12,6.53,4.03,4.24,...,11.44,10.97,30.22,0.66,1.48,11.97,12.40,8.75,11.83,10.59
2,NCU09903,2.44,2.81,5.40,4.15,2.26,5.08,4.57,1.27,2.22,...,68.48,26.35,35.37,0.97,2.09,15.79,15.86,11.41,30.68,26.72
3,NCU11134,0.76,0.31,0.22,0.14,0.47,0.32,0.21,0.13,0.15,...,0.38,0.34,20.66,0.00,0.13,0.13,0.20,0.00,0.19,0.26
4,NCU09904,2.18,1.24,1.14,1.07,1.63,2.87,2.53,0.46,0.44,...,0.96,0.34,12.31,0.38,0.17,18.59,20.32,10.27,1.22,1.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10585,NCU16025,5.46,1.44,1.10,2.54,0.47,0.82,0.98,2.25,3.19,...,0.43,1.99,237.59,3.66,7.15,4.54,4.02,4.82,2.29,4.81
10586,NCU16026,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
10587,NCU16027,50.77,15.26,15.33,27.95,6.36,11.82,11.72,24.64,29.85,...,1.22,33.38,1184.66,8.69,13.00,59.41,40.75,30.80,49.71,79.21
10588,NCU16028,61.05,21.03,17.78,34.52,14.29,16.66,16.00,35.06,36.05,...,2.70,30.90,757.43,13.13,23.55,65.50,53.66,28.06,43.04,75.57


2. 将GSM替换为fname号

In [28]:
id_map = dict(zip(df_sample["Sample Name"], df_sample["Sample ID"]))

new_columns = ["Gene id"] + [id_map[col] for col in df_gsm.columns[1:]]
df_gsm.columns = new_columns

df_fname = pd.DataFrame(df_gsm)
df_fname

,Gene id,nc006004,nc010001,nc010001,nc010001,nc010002,nc010002,nc010002,nc010003,nc010003,...,nc041004,nc004001,nc004002,nc048001,nc048002,nc011001,nc011001,nc011002,nc011002,nc011003
0,NCU10129,4.44,16.40,15.10,15.68,13.17,10.76,10.18,10.72,10.33,...,6.81,2.28,36.58,0.71,0.86,6.13,7.55,3.87,1.91,3.10
1,NCU09901,3.92,4.50,5.95,4.87,4.55,6.12,6.53,4.03,4.24,...,11.44,10.97,30.22,0.66,1.48,11.97,12.40,8.75,11.83,10.59
2,NCU09903,2.44,2.81,5.40,4.15,2.26,5.08,4.57,1.27,2.22,...,68.48,26.35,35.37,0.97,2.09,15.79,15.86,11.41,30.68,26.72
3,NCU11134,0.76,0.31,0.22,0.14,0.47,0.32,0.21,0.13,0.15,...,0.38,0.34,20.66,0.00,0.13,0.13,0.20,0.00,0.19,0.26
4,NCU09904,2.18,1.24,1.14,1.07,1.63,2.87,2.53,0.46,0.44,...,0.96,0.34,12.31,0.38,0.17,18.59,20.32,10.27,1.22,1.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10585,NCU16025,5.46,1.44,1.10,2.54,0.47,0.82,0.98,2.25,3.19,...,0.43,1.99,237.59,3.66,7.15,4.54,4.02,4.82,2.29,4.81
10586,NCU16026,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
10587,NCU16027,50.77,15.26,15.33,27.95,6.36,11.82,11.72,24.64,29.85,...,1.22,33.38,1184.66,8.69,13.00,59.41,40.75,30.80,49.71,79.21
10588,NCU16028,61.05,21.03,17.78,34.52,14.29,16.66,16.00,35.06,36.05,...,2.70,30.90,757.43,13.13,23.55,65.50,53.66,28.06,43.04,75.57


3. fname号码替换后，会出现两个列或者多列都是一个fname号，在df环境中，出现XXXX.1的情况，现在需要合并同样fname号的数据

In [29]:
df_fname = df_fname.copy()
# 解决列名重复问题
df_fname.columns = df_fname.columns.str.replace('\.\d', '')

# copy.deepcopy()函数用于创建原始对象的完全独立副本（包括嵌套对象和所有内部对象），以便在进行更改时不会影响原始对象
df_fname_new = copy.deepcopy(df_fname.iloc[:,0:1])
columns = df_fname.iloc[:,1:].columns

for col in columns:
    if len(pd.DataFrame(df_fname[col]).columns)>=2: # series转DF
        df_fname_new[col] = df_fname[col].mean(axis=1)
    else:
        df_fname_new[col] = df_fname[col]

df_fname_new = df_fname_new.round(2)
df_fname_new.to_csv(path + keyname + '_exp.tsv',sep='\t',index=False)
df_fname_new

/var/folders/9j/hx31vvg102g6r3rs7v7_4bw40000gn/T/ipykernel_48930/2957850017.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_fname.columns = df_fname.columns.str.replace('\.\d', '')
/var/folders/9j/hx31vvg102g6r3rs7v7_4bw40000gn/T/ipykernel_48930/2957850017.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fname_new[col] = df_fname[col].mean(axis=1)
/var/folders/9j/hx31vvg102g6r3rs7v7_4bw40000gn/T/ipykernel_48930/2957850017.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.c

,Gene id,nc006004,nc010001,nc010002,nc010003,nc013002,nc013003,nc013004,nc013005,nc013006,...,nc048004,nc041001,nc041002,nc041003,nc041004,nc004001,nc004002,nc011001,nc011002,nc011003
0,NCU10129,4.44,15.73,11.37,10.83,0.47,3.91,0.78,3.37,1.51,...,7.40,2.87,2.72,8.09,6.18,2.28,36.58,6.84,2.89,3.10
1,NCU09901,3.92,5.11,5.73,4.29,15.15,10.93,9.70,4.70,8.06,...,7.05,9.71,9.51,10.42,9.98,10.97,30.22,12.18,10.29,10.59
2,NCU09903,2.44,4.12,3.97,1.74,42.53,9.87,30.98,0.83,24.90,...,7.89,51.83,27.15,54.60,53.97,26.35,35.37,15.82,21.04,26.72
3,NCU11134,0.76,0.22,0.33,0.14,0.00,0.23,0.28,2.12,0.00,...,1.16,0.91,0.51,0.37,0.42,0.34,20.66,0.16,0.10,0.26
4,NCU09904,2.18,1.15,2.34,0.47,0.34,0.99,0.22,1.58,0.43,...,0.90,0.75,0.47,0.90,0.84,0.34,12.31,19.45,5.74,1.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10585,NCU16025,5.46,1.69,0.76,2.64,4.10,3.55,4.81,7.87,0.46,...,1.38,0.17,0.41,0.19,0.32,1.99,237.59,4.28,3.56,4.81
10586,NCU16026,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
10587,NCU16027,50.77,19.51,9.97,27.67,7.17,33.34,26.54,20.89,4.98,...,4.04,0.87,2.36,1.11,2.00,33.38,1184.66,50.08,40.26,79.21
10588,NCU16028,61.05,24.44,15.65,33.63,6.81,33.61,18.62,9.49,7.61,...,9.93,0.82,2.21,2.10,2.99,30.90,757.43,59.58,35.55,75.57
